In [1]:
import sys, os, time
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%matplotlib tk
import matplotlib.pyplot as plt
from tnt_util import *
from util import adict, idict, xset, collate, load, render_dict, save, Logger, seq_iterate
from tnt_setup import init_gamestate, setup_phase
from tnt_cards import load_card_decks, draw_cards
from collections import namedtuple
import random
from itertools import chain, product
from tnt_units import load_unit_rules
import tnt_setup as setup

from flask_app import *

In [2]:
ping()

'Backend active: use "init" to init game'

In [30]:
out = format_msg_to_python(init_game(debug=False, player='Axis'))

Beginning phase: Setup
Game saved


In [31]:
path = 'saves/gov_complete.json'
path = 'saves/cmd_test.json'
load(path)

Game loaded


In [32]:
G = get_G()
render_dict(G)

In [33]:
action = ('pass',)
player = 'Axis'

In [42]:
out = format_msg_to_python(get_status(player))
if 'log' in out:
    print('{} LOG'.format(player))
    print(out.log)
if 'error' in out:
    print(out.error)
print('created: {}'.format(', '.join(map(str, out.created))))
print('updated: {}'.format(', '.join(map(str, out.updated))))
print('removed: {}'.format(', '.join(map(str, out.removed))))

Axis LOG
Game loaded
West plays a card
USSR has played a bluff (investment card)
Axis has played an emergency command: Y 4
West has played a bluff (investment card)
Play order is: Axis

created: 
updated: invest_55, invest_22, action_39
removed: 


In [43]:
if 'actions' in out:
    actions = list(util.decode_actions(out.actions))
    print('ACTIONS: {}'.format(len(actions)))
    for i, a in enumerate(actions):
        print(i, a)
if 'waiting_for' in out:
    print('WAITING: {}'.format(out.waiting_for))
#render_dict(out)

ACTIONS: 136
0 (117, 'Nordkapp_Sea')
1 (119, 'Ruhr')
2 (119, 'English_Channel')
3 (128, 'Berlin')
4 (121, 'Adriatic_Sea')
5 (129, 'Ruhr')
6 (124, 'Sardinia')
7 (119, 'Berlin')
8 (134, 'Milan')
9 (116, 'Berlin')
10 (129, 'Berlin')
11 (118, 'Sicily')
12 (123, 'Taranto')
13 (132, 'Western_Mediterranean')
14 (117, 'Gulf_of_Bothnia')
15 (117, 'Norwegian_Sea')
16 (118, 'Taranto')
17 (121, 'Cyrenaica')
18 (117, 'Berlin')
19 (162, 'Rome')
20 (134, 'Taranto')
21 (116, 'Konigsberg')
22 (117, 'Greenland_Sea')
23 (129, 'Gulf_of_Bothnia')
24 (120, 'Berlin')
25 (162, 'Western_Mediterranean')
26 (119, 'Bay_of_Biscay')
27 (121, 'Tripoli')
28 (118, 'Eastern_Mediterranean')
29 (118, 'Tripoli')
30 (134, 'Western_Mediterranean')
31 (126, 'Gulf_of_Bothnia')
32 (119, 'Baltic_Sea')
33 (119, 'Irish_Sea')
34 (162, 'Milan')
35 (118, 'Crete')
36 (117, 'Bay_of_Biscay')
37 (116, 'Baltic_Sea')
38 (135, 'Berlin')
39 (121, 'Taranto')
40 (121, 'Sicily')
41 (122, 'Baltic_Sea')
42 (162, 'Taranto')
43 (132, 'Taranto')
44

In [40]:
action = actions[0]

In [41]:
out = format_msg_to_python(take_action(player, action))
if 'waiting_for' in out:
    player = out.waiting_for.pop()
    print('SWITCHING to {}'.format(player))
if 'error' in out:
    print(out.error)

SWITCHING to Axis


In [17]:
G.tiles.Algiers

{'type':'Coast', 'alligence':'French_North_Africa', 'res':0, 'pop':1, 'muster':2, 'borders':{'Gibraltar':'Strait', 'Morocco':'Mountains', 'Tunisia':'Mountains', 'Western_Mediterranean':'Coast'}, 'name':'Algiers', 'units':{148}, 'obj_type':'tile', 'visible':{'USSR', 'West', 'Axis'}}

In [63]:
out.created['604']

{'value':1, 'nation':'Yugoslavia', 'faction':'Axis', 'obj_type':'influence', 'visible':{'USSR', 'Axis', 'West'}}

In [29]:
path = save('cmd_test.json')
path

Game saved


'saves/cmd_test.json'

In [45]:
G.objects.table[116]

{ 'nationality':'Germany', 'tile':'Konigsberg', 'type':'Tank', 'obj_type':'unit', 'cv':1, 'visible':{'Axis'}}

In [10]:
# take_action('USSR', None)

In [44]:
G = get_G()

In [6]:
try:
    for player in G.game.turn_order:
        fixed[player] = [('investment_card',)]*6# + [('action_card',)]
    players = G.game.turn_order
except KeyError:
    players = ['Axis', 'USSR', 'West']
print(players)

['Axis', 'USSR', 'West']


In [7]:
for player in players:
    while True:
        out = format_msg_to_python(get_status(player))

        if 'error' in out:
            print(out.error)
            break

        if 'log' in out:
            print(out.log)

        if 'waiting_for' in out:
            print('-- {} is waiting for {}'.format(player, out.waiting_for))
            break

        if player in fixed and len(fixed[player]):
            action = fixed[player].pop()
            print('-- from {} chose {}'.format('fixed', action))
        else:
            actions = list(util.decode_actions(out.actions))
            action = random.choice(actions)
            print('-- from {} chose {}'.format(len(actions), action))
        
        out = format_msg_to_python(take_action(player, action))
        

Beginning phase: Setup
Game saved

-- from 46 chose ('Germany', 'Konigsberg', 'Carrier')

-- from 46 chose ('Germany', 'Ruhr', 'Tank')

-- from 46 chose ('Germany', 'Ruhr', 'Fleet')

-- from 46 chose ('Italy', 'Milan', 'Fleet')

-- from 46 chose ('Italy', 'Rome', 'Submarine')

-- from 46 chose ('Italy', 'Rome', 'Submarine')

-- from 43 chose ('Germany', 'Ruhr', 'Submarine')

-- from 43 chose ('Germany', 'Konigsberg', 'Carrier')

-- from 34 chose ('Germany', 'Ruhr', 'Tank')

-- from 28 chose ('Italy', 'Rome', 'AirForce')

-- from 28 chose ('Germany', 'Munich', 'Infantry')

-- from 28 chose ('Germany', 'Munich', 'Fortress')

-- from 24 chose ('Italy', 'Rome', 'Carrier')

-- from 16 chose ('Italy', 'Milan', 'AirForce')

-- from 11 chose ('Italy', 'Tripoli', 'Fleet')

-- from 11 chose ('Germany', 'Berlin', 'Submarine')

-- from 11 chose ('Italy', 'Tripoli', 'Tank')

-- from 6 chose ('Germany', 'Berlin', 'Fleet')

-- from 6 chose ('Germany', 'Berlin', 'Submarine')

-- from 6 chose ('Germany

In [6]:
# path = save('prod_complete.json')
# path

In [6]:
# sorted(5+g for g in range(50))

In [7]:
player = 'West'
draw_cards(G, 'investment', player, 20)

West draws 20 investment cards (now holding 35 cards)


In [8]:
cards = tdict({ID:G.objects.table[ID] for ID in G.players[player].hand})
render_dict(cards)

In [33]:
player = 'West'

In [10]:
player = 'USSR'

In [9]:
player = 'Axis'

In [13]:
player

'Axis'

In [11]:
out = format_msg_to_python(get_status(player))

In [12]:
if 'log' in out:
    print(out.log)
if 'error' in out:
    print(out.error)

Axis spends 1 production on building a new cadre in Venice (0 production remaining)
Axis draws 1 action cards (now holding 15 cards)
Axis draws 6 investment cards (now holding 21 cards)
Axis is done with production



In [13]:
render_dict(out)

In [ ]:
actions = list(util.decode_actions(out.actions))
actions

In [ ]:
render_dict(adict({ID:G.objects.table[ID] for ID in G.players[player].hand}))

In [ ]:
# action = random.choice(actions)
# action
action = ('factory_upgrade',)

In [13]:
player = 'West'

In [18]:
player = 'USSR'

In [11]:
player = 'Axis'

In [12]:
player

'Axis'

In [14]:
action = ('cancel',)
action = ('Britain', 'Glasgow', 'Submarine')
action = ('pass',)

In [15]:
out = format_msg_to_python(take_action(player, action))

27
0 0
{}
dict_keys([])
adict()


In [14]:
if 'log' in out:
    print(out.log)
if 'error' in out:
    print(out.error)

Game loaded
USSR draws 20 investment cards (now holding 32 cards)
Beginning phase: Government
Game saved



In [15]:
render_dict(out)